# Initialize PyImageJ

In [1]:
# Import module
import jpype
# Enable Java imports
import jpype.imports
# Pull in types
from jpype.types import *

import scyjava as sj # scyjava : Supercharged Java access from Python, see https://github.com/scijava/scyjava
import imagej

# Configurations
# NOTE: The ImageJ2 gateway is initialized through a Java Virtual Machine (JVM). If you want to configure the JVM, it must be done before initializing an ImageJ2 gateway.
# sj.config.add_option('-Xmx10g') # adjust memory available to Java
sj.config.endpoints.append('ome:formats-gpl:6.11.1')

In [2]:
Fiji_Local = r"C:\Users\confocal_microscope\Desktop\Tools\Fiji.app"

# ij = imagej.init(Fiji_Local) # Same as "ij = imagej.init(Fiji_Local, mode='headless')", PyImageJ’s default mode is headless
# ij = imagej.init(Fiji_Local, mode='gui') # GUI mode (會卡在這一行 -> blocking), for more explainations : https://pyimagej.readthedocs.io/en/latest/Initialization.html#gui-mode
ij = imagej.init(Fiji_Local, mode='interactive') # Interactive mode (可以繼續向下執行 -> non-blocking), for more explainations : https://pyimagej.readthedocs.io/en/latest/Initialization.html#interactive-mode
ij.ui().showUI() # display the Fiji GUI

print(ij.getApp().getInfo(True)) # ImageJ2 2.9.0/1.54b

ImageJ2 2.9.0/1.54b; Java 1.8.0_362 [amd64]; 203MB of 14542MB


# Import Customized modules and Create ImageJ Object

In [3]:
import os
import sys
import re
from typing import TextIO
from datetime import datetime
from glob import glob

import json
import logging
from skimage.io import imread, imshow, imsave


sys.path.append(r"C:\Users\confocal_microscope\Desktop\ZebraFish_AP_POS\modules") # add path to scan customized module

from logger import init_logger
from lifcheck import check_image_name

In [4]:
# Bio-Format Reader
loci = jpype.JPackage("loci")
loci.common.DebugTools.setRootLevel("ERROR")
Reader = loci.formats.ImageReader()

In [5]:
log = init_logger(r"Check Lif Name")

# Run

In [4]:
nas_data_root = r"C:\Users\confocal_microscope\Desktop\{NAS_DL}_Academia_Sinica_Data" # constant
ap_data_dir = r"{20230301_Update}_Zebrafish_A_P_strategies"
image_type = "BrightField_RAW" # BrightField_RAW / palmskin_RGB_RAW
upload_date = "Upload_20230301"

raw_lif_dir = os.path.join(nas_data_root, ap_data_dir, image_type, upload_date)

In [ ]:
lif_path_list = glob(os.path.normpath(f"{raw_lif_dir}/*.lif"))
log.info(f'lif_path_list [ found {len(lif_path_list)} files ]: {lif_path_list},  \n')


for i, lif_path in enumerate(lif_path_list):
    
    log.info(f"Processing ... {i+1}/{len(lif_path_list)}")
    log.info(f'LIF_FILE : {lif_path}')
    
    Reader.setId(lif_path)
    seriesCount = Reader.getSeriesCount()
    
    for idx in range(seriesCount):
        
        SeriesNum = idx+1
        
        ij.IJ.run("Bio-Formats Importer", f"open='{lif_path}' color_mode=Default rois_import=[ROI manager] view=Hyperstack stack_order=XYCZT series_{SeriesNum}")
        img = ij.WindowManager.getCurrentImage() # get image, <java class 'ij.ImagePlus'>
        # dump_info(img)
        # img.show()
        img.hide()
        
        
        # Get names and image dimensions
        
        # Consociate "palmskin_RGB_RAW" file_name
        file_name = lif_path.split(os.sep)[-1].split(".")[0]
        file_name_list = re.split(" |_|-", file_name)
        assert len(file_name_list) == 4, f"file_name format error, current : '{file_name}', expect like : '20221125_AI005_palmskin_10dpf.lif'"
        file_name = "_".join(file_name_list)
        
        # Get Prop("Image name")
        image_name = img.getProp("Image name")
        image_name = str(image_name)
        
        # Check format of 'image_name'
        image_name_list = re.split(" |_|-", image_name)
        
        
        if image_type == "BrightField_RAW":
            name_format_list = re.findall("[BF]", image_name)
            # print(name_format_list)
            if len(name_format_list) > 0: failed_cnt, check_dict = check_image_name(image_name_list, "new_bf")
            else: failed_cnt, check_dict = check_image_name(image_name_list, "old_bf")
        
            # Consociate image_name
            if (check_dict["    format_and_type     "] == "old_bf") and (failed_cnt == 0): # name check passed, generate new name
                image_name_list.pop(3) # palmskin (old name only)
                image_name_list.pop(3) # [num]dpf (old name only)
                image_name_list.append("BF")
            image_name = "_".join(image_name_list)
        
        
        if image_type == "palmskin_RGB_RAW":
            name_format_list = re.findall("[RGB]", image_name)
            # print(name_format_list)
            if len(name_format_list) > 0: failed_cnt, check_dict = check_image_name(image_name_list, "new_rgb")
            else: failed_cnt, check_dict = check_image_name(image_name_list, "old_rgb")

            # Consociate image_name
            if (check_dict["    format_and_type     "] == "old_rgb") and (failed_cnt == 0): # name check passed, generate new name
                image_name_list.pop(3) # palmskin (old name only)
                image_name_list.pop(3) # [num]dpf (old name only)
                image_name_list.append("RGB")
            image_name = "_".join(image_name_list)
        
        
        # Combine 2 names above
        seN = f"{file_name} - {image_name}"
        
        # Get dimension info
        img_dimensions = img.getDimensions()
        
        log.info(f"series {SeriesNum:{len(str(seriesCount))}}/{seriesCount} : '{seN}' , Dimensions : {img_dimensions} ( width, height, channels, slices, frames )")
        
        # Print ERRORs when checking 'image_name'
        if failed_cnt == -1 : log.info(check_dict["  len(image_name_list)  "])
        if failed_cnt  >  0 : log.info(check_dict["     failed warning     "])
        
        
    log.info("\n")


log.info(" -- finished --")